In [1]:
import re
from lightrag.core import Component, Generator
from lightrag.components.model_client import OpenAIClient
from lightrag.components.model_client import GroqAPIClient
from lightrag.utils import setup_env # make sure you have a .env file with OPENAI_API_KEY and GROQ_API_KEY

In [2]:
template_doc = r"""<SYS> You are a doctor </SYS> User: {{input_str}}"""

Let's turn on the library log to help with debugging.

In [3]:
from lightrag.utils import enable_library_logging
enable_library_logging()

In [4]:
#Toy example

class DocQA(Component):
    def __init__(self):
        super(DocQA, self).__init__()
        self.doc = Generator(
            template=template_doc,
            model_client=OpenAIClient(),
            model_kwargs={"model": "gpt-3.5-turbo"},
        )

    def call(self, query: str) -> str:
        return self.doc(prompt_kwargs={"input_str": query}).data
    

In [5]:
doc = DocQA()
doc

2024-06-14 17:41:30 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: []


DocQA(
  (doc): Generator(
    model_kwargs={'model': 'gpt-3.5-turbo'}, 
    (prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}})
    (model_client): OpenAIClient()
  )
)

In [6]:
# states
states = doc.to_dict()
print(states)
doc.__dict__

{'type': 'DocQA', 'data': {'_components': {'_ordered_dict': True, 'data': [('doc', {'type': 'Generator', 'data': {'_components': {'_ordered_dict': True, 'data': [('prompt', {'type': 'Prompt', 'data': {'_components': {'_ordered_dict': True, 'data': []}, '_parameters': {'_ordered_dict': True, 'data': []}, 'training': False, 'template': '<SYS> You are a doctor </SYS> User: {{input_str}}', 'prompt_variables': [], 'preset_prompt_kwargs': {}}}), ('model_client', {'type': 'OpenAIClient', 'data': {'_components': {'_ordered_dict': True, 'data': []}, '_parameters': {'_ordered_dict': True, 'data': []}, 'training': False, '_api_key': None}})]}, '_parameters': {'_ordered_dict': True, 'data': []}, 'training': False, 'template': '<SYS> You are a doctor </SYS> User: {{input_str}}', 'preset_prompt_kwargs': {}, 'model_kwargs': {'model': 'gpt-3.5-turbo'}, 'output_processors': None, '_trainable_params': []}})]}, '_parameters': {'_ordered_dict': True, 'data': []}, 'training': False}}


{'_components': OrderedDict([('doc',
               Generator(
                 model_kwargs={'model': 'gpt-3.5-turbo'}, 
                 (prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}})
                 (model_client): OpenAIClient()
               ))]),
 '_parameters': OrderedDict(),
 'training': False}

In [7]:
# restore the states
doc2 = DocQA.from_dict(states)
# print(doc2.call("What is the capital of France?"))
doc2.__dict__
# doc2.to_dict()

{'_components': OrderedDict([('doc',
               Generator(
                 model_kwargs={'model': 'gpt-3.5-turbo'}, 
                 (prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}})
                 (model_client): OpenAIClient()
               ))]),
 '_parameters': OrderedDict(),
 'training': False}

In [8]:
doc2.to_dict() == doc.to_dict()
doc2.to_dict()

{'type': 'DocQA',
 'data': {'_components': {'_ordered_dict': True,
   'data': [('doc',
     {'type': 'Generator',
      'data': {'_components': {'_ordered_dict': True,
        'data': [('prompt',
          {'type': 'Prompt',
           'data': {'_components': {'_ordered_dict': True, 'data': []},
            '_parameters': {'_ordered_dict': True, 'data': []},
            'training': False,
            'template': '<SYS> You are a doctor </SYS> User: {{input_str}}',
            'prompt_variables': [],
            'preset_prompt_kwargs': {}}}),
         ('model_client',
          {'type': 'OpenAIClient',
           'data': {'_components': {'_ordered_dict': True, 'data': []},
            '_parameters': {'_ordered_dict': True, 'data': []},
            'training': False,
            '_api_key': None}})]},
       '_parameters': {'_ordered_dict': True, 'data': []},
       'training': False,
       'template': '<SYS> You are a doctor </SYS> User: {{input_str}}',
       'preset_prompt_kwargs': {

In [18]:
# pickle to states
import pickle
# from collections import OrderedDict
# from openai import OpenAI # cant pickle this

# class DummpyDocQA():
#     a = OrderedDict()
#     def __init__(self):
#         self.dummpy = 1
#         self.client = OpenAI()

# doc_dummy = DummpyDocQA()
with open("doc.pkl", "wb") as f:
    pickle.dump(doc.to_dict(), f)

In [12]:
# save the serialized states to a file
from lightrag.utils.serialization import save_pickle, save_json
states = doc.to_dict()
# save_json(states, "doc.json")
save_pickle(states, "doc.pkl")

# load the serialized states from a file
from lightrag.utils.serialization import load_pickle, load_json
states_loaded = load_pickle("doc.pkl")
# states_loaded = load_json("doc.json")

states_loaded == states

doc3 = DocQA.from_dict(states_loaded)


In [13]:
doc3
doc3.call("What is the capital of France?")

2024-06-14 17:42:48 - INFO - [generator.py:199:call] - prompt_kwargs: {'input_str': 'What is the capital of France?'}
2024-06-14 17:42:48 - INFO - [generator.py:200:call] - model_kwargs: {}
2024-06-14 17:42:48 - WARNING - [prompt_builder.py:120:compose_prompt_kwargs] - Key input_str does not exist in the prompt_kwargs.
2024-06-14 17:42:48 - INFO - [openai_client.py:139:call] - api_kwargs: {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '<SYS> You are a doctor </SYS> User: What is the capital of France?'}]}
2024-06-14 17:42:48 - INFO - [_client.py:1026:_send_single_request] - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-14 17:42:48 - INFO - [generator.py:208:call] - output: GeneratorOutput(data='The capital of France is Paris.', error=None, usage=None, raw_response='The capital of France is Paris.')


'The capital of France is Paris.'

In [ ]:
print(doc("What is the best treatment for headache?"))

2024-06-14 17:12:51 - INFO - [generator.py:199:call] - prompt_kwargs: {'input_str': 'What is the best treatment for headache?'}
2024-06-14 17:12:51 - INFO - [generator.py:200:call] - model_kwargs: {}
2024-06-14 17:12:51 - INFO - [openai_client.py:140:call] - api_kwargs: {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '<SYS> You are a doctor </SYS> User: What is the best treatment for headache?'}]}
2024-06-14 17:12:54 - INFO - [_client.py:1026:_send_single_request] - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-14 17:12:54 - INFO - [generator.py:208:call] - output: GeneratorOutput(data='As a doctor, the best treatment for a headache depends on the cause of the headache. In general, some common treatments for headaches include:\n\n1. Over-the-counter pain relievers such as acetaminophen, ibuprofen, or aspirin\n2. Rest and relaxation in a quiet, dark room\n3. Hydration\n4. Applying a cold or warm compress to the forehead or 

In [ ]:
print(doc2("What is the best treatment for headache?"))

2024-06-14 17:12:54 - INFO - [generator.py:199:call] - prompt_kwargs: {'input_str': 'What is the best treatment for headache?'}
2024-06-14 17:12:54 - INFO - [generator.py:200:call] - model_kwargs: {}
2024-06-14 17:12:54 - INFO - [openai_client.py:140:call] - api_kwargs: {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '<SYS> You are a doctor </SYS> User: What is the best treatment for headache?'}]}
2024-06-14 17:12:56 - INFO - [_client.py:1026:_send_single_request] - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-14 17:12:56 - INFO - [generator.py:208:call] - output: GeneratorOutput(data='As a doctor, the best treatment for a headache can depend on the cause of the headache. If the headache is mild and infrequent, over-the-counter pain relievers such as ibuprofen or acetaminophen can help. Additionally, getting enough rest, staying hydrated, and practicing relaxation techniques such as deep breathing exercises or meditation 

In [ ]:
# list other subcomponents

for subcomponent in doc.named_components():
    print(subcomponent)

('', DocQA(
  (doc): Generator(
    model_kwargs={'model': 'gpt-3.5-turbo'}, 
    (prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
    (model_client): OpenAIClient()
  )
))
('doc', Generator(
  model_kwargs={'model': 'gpt-3.5-turbo'}, 
  (prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
  (model_client): OpenAIClient()
))
('doc.prompt', Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str']))
('doc.model_client', OpenAIClient())


Let's add a parameter

In [ ]:
from lightrag.core.parameter import Parameter

doc.register_parameter("demo", param=Parameter(data="demo"))

In [ ]:
# list all parameters
for param in doc.named_parameters():
    print(param)

('demo', Parameter: demo)


In [ ]:
doc.to_dict()

{'type': 'DocQA',
 'data': {'_components': {'_ordered_dict': True,
   'data': [('doc',
     {'type': 'Generator',
      'data': {'_components': {'_ordered_dict': True,
        'data': [('prompt',
          {'type': 'Prompt',
           'data': {'_components': {'_ordered_dict': True, 'data': []},
            '_parameters': {'_ordered_dict': True, 'data': []},
            'training': False,
            '_template_string': '<SYS> You are a doctor </SYS> User: {{input_str}}',
            'template': <Template memory:15f114f50>,
            'prompt_variables': ['input_str'],
            'preset_prompt_kwargs': {}}}),
         ('model_client',
          {'type': 'OpenAIClient',
           'data': {'_components': {'_ordered_dict': True, 'data': []},
            '_parameters': {'_ordered_dict': True, 'data': []},
            'training': False,
            'sync_client': <openai.OpenAI at 0x15cfc1cd0>,
            'async_client': None,
            '_api_key': None}})]},
       '_parameters': {'

In [ ]:
from lightrag.utils.serialization import save_json

save_json(doc.to_dict(), "doc.json")

In [ ]:
doc.state_dict()

OrderedDict([('demo', Parameter: demo)])

In [ ]:
doc.call("What is the best treatment for a cold?")

2024-06-14 17:12:56 - INFO - [generator.py:199:call] - prompt_kwargs: {'input_str': 'What is the best treatment for a cold?'}
2024-06-14 17:12:56 - INFO - [generator.py:200:call] - model_kwargs: {}
2024-06-14 17:12:56 - INFO - [openai_client.py:140:call] - api_kwargs: {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '<SYS> You are a doctor </SYS> User: What is the best treatment for a cold?'}]}
2024-06-14 17:12:57 - INFO - [_client.py:1026:_send_single_request] - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-14 17:12:57 - INFO - [generator.py:208:call] - output: GeneratorOutput(data="As a doctor, I recommend a combination of rest, staying hydrated, over-the-counter cold medications (such as decongestants or pain relievers), throat lozenges, and steam inhalation. It's also important to eat a balanced diet, get plenty of rest, and avoid close contact with others to prevent spreading the cold. If symptoms persist or worsen, it

"As a doctor, I recommend a combination of rest, staying hydrated, over-the-counter cold medications (such as decongestants or pain relievers), throat lozenges, and steam inhalation. It's also important to eat a balanced diet, get plenty of rest, and avoid close contact with others to prevent spreading the cold. If symptoms persist or worsen, it's best to consult with a healthcare provider for further evaluation and treatment."

In [ ]:
from lightrag.core.component import FunComponent

def add_one(x):
    return x + 1

fun_component = FunComponent(add_one)
print(fun_component(1))  
print(type(fun_component))  

# output:
# 2
# <class 'core.component.FunComponent'>

2
<class 'lightrag.core.component.FunComponent'>


In [ ]:
from lightrag.core.component import fun_to_component 

fun_component = fun_to_component(add_one)
print(fun_component(1))
print(type(fun_component))

# output:
# 2
# <class 'lightrag.core.component.AddOneComponent'>

2
<class 'lightrag.core.component.AddOneComponent'>


In [ ]:
# use it as a decorator
@fun_to_component
def add_one(x):
    return x + 1

print(add_one(1))
print(type(add_one))

# output:
# 2
# <class 'lightrag.core.component.AddOneComponent'>

2
<class 'lightrag.core.component.AddOneComponent'>


In [ ]:
from lightrag.core.component import Sequential

@fun_to_component
def enhance_query(query:str) -> str:
    return query + "Please be concise and only list the top treatments."

seq = Sequential(enhance_query, doc)

query = "What is the best treatment for headache?"
print(seq(query))

2024-06-14 17:12:57 - INFO - [generator.py:199:call] - prompt_kwargs: {'input_str': 'What is the best treatment for headache?Please be concise and only list the top treatments.'}
2024-06-14 17:12:57 - INFO - [generator.py:200:call] - model_kwargs: {}
2024-06-14 17:12:57 - INFO - [openai_client.py:140:call] - api_kwargs: {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '<SYS> You are a doctor </SYS> User: What is the best treatment for headache?Please be concise and only list the top treatments.'}]}
2024-06-14 17:12:58 - INFO - [_client.py:1026:_send_single_request] - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-14 17:12:58 - INFO - [generator.py:208:call] - output: GeneratorOutput(data='1. Over-the-counter pain relievers such as ibuprofen or acetaminophen\n2. Stay hydrated and rest\n3. Apply a cold compress to the forehead\n4. Practice relaxation techniques such as deep breathing or meditation', error=None, usage=None, raw

In [ ]:
# sequential with just a function, will raise error
# def enhance_query(query:str) -> str:
#     return query + "Please be concise and only list the top treatments."
# seq2 = Sequential(enhance_query, doc)
# print(seq2(query))
# print(seq2)

In [ ]:
seq

Sequential(
  (0): EnhanceQueryComponent()
  (1): DocQA(
    (doc): Generator(
      model_kwargs={'model': 'gpt-3.5-turbo'}, 
      (prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
      (model_client): OpenAIClient()
    )
  )
)

# TODO: LLM for single choices